In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from transformers import BitsAndBytesConfig

In [ ]:
train_df = pd.read_csv('train_combined_llama.csv')
test_df = pd.read_csv('val_combined_llama.csv')

combine_train_val_for_training = 'method1' #this corresponds to method 1 and method 2 described in the paper

if combine_train_val_for_training=='method1':
    train_df_combined = pd.concat([train_df,test_df[:-1000]])
    test_df_combined = test_df[-1000:]
else:
    train_df_combined = train_df
    test_df_combined = test_df

In [ ]:
train_ds = Dataset.from_dict(train_df_combined)
test_ds = Dataset.from_dict(test_df_combined)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer_path = 'meta-llama/Llama-2-13b-chat-hf'
model_path = 'meta-llama/Llama-2-13b-chat-hf'

#change cache_dir path if you already have the meta-llama/Llama-2-13b-chat-hf downloaded
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path,cache_dir="./model_cache_folder/")
model = AutoModelForCausalLM.from_pretrained(model_path,cache_dir="./model_cache_folder/",
                                            device_map='auto',quantization_config=bnb_config,use_cache=False)

In [ ]:
tokenizer.pad_token = '给'
tokenizer.padding_side = "right"

In [ ]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=4,
        bias="none",
        task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)

In [ ]:
args = TrainingArguments(
    output_dir="./llama2_13b_3ep",
    num_train_epochs=3,
    logging_strategy='steps',
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 4,
    save_strategy="epoch",
    evaluation_strategy='epoch',
    gradient_accumulation_steps=2,
    logging_steps=200,
    save_steps=1000,
    fp16=True,
    optim="paged_adamw_32bit",
    eval_steps=1000,
    learning_rate=2e-5,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

In [ ]:
max_seq_length = 4096

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    dataset_text_field="prompt",
    eval_dataset = test_ds,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=False,
    args=args,
)

In [ ]:
trainer.train()
trainer.save_model()